## 0. Requirement

In [304]:
## jalankan ini dahulu sebelum running
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [305]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [306]:
## check if there is a response, if it's 200, we are good to go
s = requests.Session()
url = 'https://news.detik.com/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
}
response = s.get(url, headers=headers, timeout=1)
print(response.status_code)

200


## 1. HTML and CSS Selector

In [307]:
soup = BeautifulSoup(response.content, 'html.parser')

In [308]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="id-ID">
<head>
<script src="https://iat.detiknetwork.com/ip-information.js" type="text/javascript"></script>
<link href="https://awscdn.detik.net.id" rel="preconnect"/><link crossorigin="" href="https://awscdn.detik.net.id" rel="preconnect"/><link href="https://cdn.detik.net.id" rel="preconnect"/><link crossorigin="" href="https://cdn.detik.net.id" rel="preconnect"/> <link href="https://cdn.detik.net.id" rel="dns-prefetch"><link href="https://akcdn.detik.net.id" rel="dns-prefetch"><link href="https://cdnv.detik.net.id" rel="dns-prefetch"><link href="https://connect.detik.com" rel="dns-prefetch"><link href="https://newrevive.detik.com" rel="dns-prefetch"/><link href="https://cdnstatic.detik.com" rel="dns-prefetch"/><link href="https://analytic.detik.com" rel="dns-prefetch"/><link href="https://www.googletagmanager.com" rel="dns-prefetch"/><link href="https://securepubads.g.doubleclick.net" rel="dns-prefetch"/><link href="https:/

In [309]:
soup.find('h3', class_= 'media__title')

<h3 class="media__title">
<a class="media__link" dtr-act="artikel" dtr-evt="newsfeed" dtr-id="7935719" dtr-idx="1" dtr-sec="" dtr-ttl="Video: PM Malaysia Anwar Ibrahim Buka KTT ASEAN-GCC" href="https://20.detik.com/detikupdate/20250527-250527104/video-pm-malaysia-anwar-ibrahim-buka-ktt-asean-gcc" onclick="_pt(this)">
<div class="replace_title">
                        Video: PM Malaysia Anwar Ibrahim Buka KTT ASEAN-GCC                    </div>
</a>
</h3>

In [310]:
soup.find('h3', class_= 'media__title').get_text()

'\n\n\n                        Video: PM Malaysia Anwar Ibrahim Buka KTT ASEAN-GCC                    \n\n'

In [311]:
soup.find('h3', class_= 'media__title').get_text(strip=True)

'Video: PM Malaysia Anwar Ibrahim Buka KTT ASEAN-GCC'

In [312]:
soup.find('div', class_="media__text ph-item").get_text()

'\n\n\n\n                        Video: PM Malaysia Anwar Ibrahim Buka KTT ASEAN-GCC                    \n\n\n\n1 menit yang lalu \n'

In [313]:
soup.find_all('div', class_="media__text ph-item")

[<div class="media__text ph-item">
 <h3 class="media__title">
 <a class="media__link" dtr-act="artikel" dtr-evt="newsfeed" dtr-id="7935719" dtr-idx="1" dtr-sec="" dtr-ttl="Video: PM Malaysia Anwar Ibrahim Buka KTT ASEAN-GCC" href="https://20.detik.com/detikupdate/20250527-250527104/video-pm-malaysia-anwar-ibrahim-buka-ktt-asean-gcc" onclick="_pt(this)">
 <div class="replace_title">
                         Video: PM Malaysia Anwar Ibrahim Buka KTT ASEAN-GCC                    </div>
 </a>
 </h3>
 <div class="media__date">
 <span d-time="1748342883" title="Selasa, 27 Mei  2025 17:48 WIB">1 menit yang lalu</span> </div>
 </div>,
 <div class="media__text ph-item">
 <h3 class="media__title">
 <a class="media__link" dtr-act="artikel" dtr-evt="newsfeed" dtr-id="7935715" dtr-idx="2" dtr-sec="" dtr-ttl="Video: Stop Bakar Sampah Sembarangan! Ada Aturan Hukumnya" href="https://20.detik.com/detikupdate/20250527-250527103/video-stop-bakar-sampah-sembarangan-ada-aturan-hukumnya" onclick="_pt(this)"

In [314]:
beritas = soup.find_all('div', class_="media__text")
hasil = []

for berita in beritas:
    title = berita.find('div', class_="replace_title")
    upload = berita.find('div', class_="media__date")
    if title and upload:
        hasil.append({
            "judul_berita": title.get_text(strip=True),
            "waktu_upload": upload.get_text(strip=True)
        }
        if title and upload else None
        )

df = pd.DataFrame(hasil)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   judul_berita  50 non-null     object
 1   waktu_upload  50 non-null     object
dtypes: object(2)
memory usage: 932.0+ bytes


In [315]:
df.head()

,judul_berita,waktu_upload
0,Video: PM Malaysia Anwar Ibrahim Buka KTT ASEA...,1 menit yang lalu
1,Video: Stop Bakar Sampah Sembarangan! Ada Atur...,2 menit yang lalu
2,Video: Pasutri Anak di Lombok Penuhi Panggilan...,3 menit yang lalu
3,"Hadiri KTT ASEAN, Sultan Brunei Dibawa ke RS M...",5 menit yang lalu
4,Deretan 11 Apartemen Eks Dirut Taspen Diduga d...,7 menit yang lalu


## 2. Xpath

In [318]:
import requests
from lxml import html

url = "https://news.detik.com/"
response = requests.get(url)
tree = html.fromstring(response.content)

In [319]:
tree.xpath('//h3[@class="media__title"]/a/text()')

['\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',
 '\n                    ',
 '\n                ',


In [320]:
list_judul = tree.xpath('//h3[@class="media__title"]/a/text()')
juduls = [judul.strip() for judul in list_judul if judul.strip()]
juduls

['Pendaftaran Adhyaksa Awards Diperpanjang hingga 30 Juni 2025, Usulkan Kandidatmu!',
 'Pesan Pemenang Adhyaksa Awards 2024: Lakukan dengan Hati dan Konsistensi',
 'Beragam Cerita Teladan dari 15 Polisi Kandidat Hoegeng Awards 2025',
 'Cerita Dewan Pakar Hati-hati Pilih 15 Kandidat Hoegeng Awards 2025',
 'grup facebook fantasi sedarah',
 'hasto kristiyanto',
 'sritex',
 'ijazah jokowi',
 'hoegeng awards',
 'Sosok Istri Presiden Prancis yang Viral Toyor Suaminya',
 'Jerman Bilang Serangan Israel di Gaza Bukan Lagi Perang Lawan Hamas',
 'Jaksa Agung Turun Gunung Jenguk Pegawai Kejagung yang Dibacok di Depok',
 'Christiano Tarigan Pengemudi BMW Jadi Tersangka Tewasnya Argo Mahasiswa UGM',
 'Polisi Gerebek Pesta Seks Gay di Hotel Jaksel, 9 Orang Diringkus!',
 'Jokowi: Saya Sedih Jika Proses Hukum soal Ijazah ke Tahap Berikutnya, Kasihan',
 'Bareskrim Tampilkan Foto-foto Saat Jokowi KKN hingga Wisuda di UGM',
 'Tak Ada Unsur Pidana, Bareskrim Setop Penyelidikan Isu Ijazah Palsu Jokowi']

In [321]:
list_upload = tree.xpath('.//div[@class="media__date"]/span//text()')
list_upload

['1 menit yang lalu',
 '2 menit yang lalu',
 '3 menit yang lalu',
 '5 menit yang lalu',
 '7 menit yang lalu',
 '10 menit yang lalu',
 '11 menit yang lalu',
 '16 menit yang lalu',
 '19 menit yang lalu',
 '19 menit yang lalu',
 '19 menit yang lalu',
 '21 menit yang lalu',
 '21 menit yang lalu',
 '24 menit yang lalu',
 '24 menit yang lalu',
 '31 menit yang lalu',
 '31 menit yang lalu',
 '31 menit yang lalu',
 '33 menit yang lalu',
 '34 menit yang lalu',
 '37 menit yang lalu',
 '39 menit yang lalu',
 '43 menit yang lalu',
 '45 menit yang lalu',
 '46 menit yang lalu',
 '46 menit yang lalu',
 '47 menit yang lalu',
 '48 menit yang lalu',
 '53 menit yang lalu',
 '54 menit yang lalu',
 '55 menit yang lalu',
 '59 menit yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam yang lalu',
 '1 jam y

In [322]:
link_raw = tree.xpath('.//h3[@class="media__title"]/a/@href')
link_raw

['https://20.detik.com/detikupdate/20250527-250527104/video-pm-malaysia-anwar-ibrahim-buka-ktt-asean-gcc',
 'https://20.detik.com/detikupdate/20250527-250527103/video-stop-bakar-sampah-sembarangan-ada-aturan-hukumnya',
 'https://20.detik.com/detikupdate/20250527-250527105/video-pasutri-anak-di-lombok-penuhi-panggilan-polisi-soal-pernikahan-dini',
 'https://news.detik.com/internasional/d-7935705/hadiri-ktt-asean-sultan-brunei-dibawa-ke-rs-malaysia-karena-kelelahan',
 'https://news.detik.com/berita/d-7935695/deretan-11-apartemen-eks-dirut-taspen-diduga-dari-korupsi-termahal-rp-10-m',
 'https://news.detik.com/berita/d-7935690/mei-banjir-tanggal-merah-masih-ada-2-kali-long-weekend-lagi-di-juni-2025',
 'https://news.detik.com/berita/d-7935689/bupati-serang-bakal-tempati-rumah-pribadi-tak-akan-minta-anggaran-sewa',
 'https://news.detik.com/berita/d-7935682/serahkan-rpjmd-banten-2025-2029-andra-soni-harap-segera-disahkan-dprd',
 'https://news.detik.com/abc-australia/d-7935677/dunia-hari-ini-m

In [345]:
import requests
from lxml import html
import pandas as pd

url = "https://news.detik.com/"
response = requests.get(url)
tree = html.fromstring(response.content)

beritas = tree.xpath('//div[contains(@class, "media__text")]')

hasil = []
for berita in beritas:
        title_raw = berita.xpath('//h3[@class="media__title"]/a/text()')
        # title = title_raw[0].strip() if title_raw else ""
        title = [t.strip() for t in title_raw if t.strip()] if title else ""

        upload_raw = berita.xpath('.//div[contains(@class, "media__date")]/span//text()')
        upload = upload_raw[0].strip() if upload_raw else ""

        link_raw = berita.xpath('.//h3[contains(@class, "media__title")]/a/@href')
        link = link_raw[0].strip() if link_raw else ""
        
        # Skip jika semua kosong
        if not title and not upload and not link:
            continue

        hasil.append({
            "judul_berita": title,
            "waktu_upload": upload,
            "tautan_berita": link
        })

df = pd.DataFrame(hasil)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   judul_berita   68 non-null     object
 1   waktu_upload   68 non-null     object
 2   tautan_berita  68 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB
